# Summarize Data
11/17/23<br>
This notebook analyzes Divvy station status over time and organizes results for analysis<br>

# import libraries and set up analysis

In [71]:
import pandas as pd
from datetime import datetime

#highlight cell where user can edit filename
import ipywidgets as widgets
from IPython.display import display, HTML

### specify file name

In [78]:
#all this fancy code to create a yellah-highlighted textbox, emphasizing that I should give the file a meaningful name
filename_base = widgets.Text(
    description='Filename (describe the dataset):',
    value='2023_11_17_10hours',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# Display the widget
display(filename_base)

# Use HTML to inject custom CSS for the text widget
style = """
<style>
    .widget-text input {
        background-color: yellow !important;
    }
</style>
"""

display(HTML(style))

Text(value='2023_11_17_10hours', description='Filename (describe the dataset):', layout=Layout(width='50%'), s…

In [79]:
#append a timestamp to prevent overwriting files
now = datetime.now()
current_timestamp=int(now.timestamp())
current_timestamp
file_suffix = f"{filename_base.value}_{current_timestamp}.csv"
file_suffix

'2023_11_17_10hours_1700255877.csv'

### assign binning for values
for this analysis, the categories will be colored red-orange-yellow-green-blue based on the bins

In [80]:
def assign_bin(value):
    if value >= 90:
        return '90% to 100%'
    elif 75 <= value < 90:
        return '75% to 89%'
    elif 25 <= value < 75:
        return '25% to 74%'
    elif 10 <= value < 25:
        return '10% to 24%'
    else:
        return '0% to 9%'

# import data

In [51]:
df = pd.read_csv("../results/station-status-hourly-2023-11-17.csv")

In [52]:
df.keys()

Index(['num_bikes_disabled', 'num_scooters_available', 'num_docks_available',
       'num_docks_disabled', 'is_installed', 'is_renting',
       'num_bikes_available', 'num_scooters_unavailable',
       'num_ebikes_available', 'station_id', 'is_returning', 'time_reported',
       'n_classic', 'n_electric', 'n_scooters', 'time_retrieved',
       'is_no_classic', 'is_no_docks', 'is_problem_station', 'name', 'lat',
       'lon', 'capacity', 'is_public_rack'],
      dtype='object')

In [53]:
df_summary = result = df.groupby(['station_id','name','capacity','lat','lon']).agg(
    no_classic_count=('is_no_classic', 'sum'),       # Count of is_no_class=True
    no_docks_count=('is_no_docks', 'sum'),       # Count of is_no_docks=True
    problem_count=('is_problem_station', 'sum'),       # either no docks or no classic bikes
    total_count=('station_id', 'size')           # Total count of records per station_id
)
df_summary["pct_no_classic"]= 100*station_summary["no_classic_count"]/station_summary["total_count"]
df_summary["pct_no_docks"]= 100*station_summary["no_docks_count"]/station_summary["total_count"]
df_summary["pct_problem"]= 100*station_summary["problem_count"]/station_summary["total_count"]
df_summary["bin_no_classic"]= df_summary['pct_no_classic'].apply(assign_bin)
df_summary["bin_no_docks"]= df_summary['pct_no_docks'].apply(assign_bin)
df_summary["bin_problem"]= df_summary['pct_problem'].apply(assign_bin)
df_summary

,,,,,no_classic_count,no_docks_count,problem_count,total_count,pct_no_classic,pct_no_docks,pct_problem,bin_no_classic,bin_no_docks,bin_problem
station_id,name,capacity,lat,lon,,,,,,,,,,
03a1610c-3ba0-4b85-b512-bc26791cb7ee,Eberhart Ave & 91st St,15,41.729285,-87.611032,0,0,0,10,0.0,0.0,0.0,0% to 9%,0% to 9%,0% to 9%
080ab938-2c1b-4fcf-a83d-74c5ae6ff305,Cottage Grove Ave & 111th Pl,15,41.691710,-87.610002,0,0,0,10,0.0,0.0,0.0,0% to 9%,0% to 9%,0% to 9%
0eeb6baf-c653-4a1a-af17-99f686116b89,Lawndale Ave & 111th St,15,41.691319,-87.712886,0,0,0,10,0.0,0.0,0.0,0% to 9%,0% to 9%,0% to 9%
0f9b826b-8885-4c25-96fd-9e2c2074c34e,Western Ave & 104th St,15,41.704587,-87.681203,0,0,0,10,0.0,0.0,0.0,0% to 9%,0% to 9%,0% to 9%
10327d47-3c3a-433e-9e38-80638da64232,Indiana Ave & 103rd St,15,41.707170,-87.618472,0,0,0,10,0.0,0.0,0.0,0% to 9%,0% to 9%,0% to 9%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
da9e6779-5056-4d45-bae3-9dc325ed2807,Hale Ave & 107th St,11,41.699206,-87.668919,0,0,0,10,0.0,0.0,0.0,0% to 9%,0% to 9%,0% to 9%
e63300bd-4ca3-4255-adfd-522289d68b8a,Loomis Blvd & 84th St,15,41.741487,-87.658410,0,0,0,10,0.0,0.0,0.0,0% to 9%,0% to 9%,0% to 9%
ee925757-31ce-41ad-9bb7-73b1bb363616,Bradley Park,11,41.717878,-87.564592,0,0,0,10,0.0,0.0,0.0,0% to 9%,0% to 9%,0% to 9%


# Export for Flourish

In [54]:
df_summary.to_csv(f"../results/stations_summary_{file_suffix}")

# Preview Some Data

In [21]:
# top ten worst stations
station_summary.sort_values('pct_problem', ascending=False).head(10)

,no_classic_count,no_docks_count,problem_count,total_count,pct_no_classic,pct_no_docks,pct_problem
station_id,,,,,,,
1872723419889143870,10,0,10,10,100.0,0.0,100.0
1594046400988140226,10,0,10,10,100.0,0.0,100.0
1594046370923369114,10,0,10,10,100.0,0.0,100.0
1594046379513303716,10,0,10,10,100.0,0.0,100.0
1594046379513303720,10,0,10,10,100.0,0.0,100.0
1594046383808271020,10,0,10,10,100.0,0.0,100.0
1594046383808271024,10,0,10,10,100.0,0.0,100.0
1594046388103238324,10,0,10,10,100.0,0.0,100.0
1594046388103238326,10,0,10,10,100.0,0.0,100.0


In [22]:
# top ten worst stations
station_summary.sort_values('pct_no_docks', ascending=False).head(10)

,no_classic_count,no_docks_count,problem_count,total_count,pct_no_classic,pct_no_docks,pct_problem
station_id,,,,,,,
1594046448232780546,10,10,10,10,100.0,100.0,100.0
a3b1e588-a135-11e9-9cda-0a87ae2ba916,0,10,10,10,0.0,100.0,100.0
61695579-2f75-4090-a071-c76079b5fb5d,0,10,10,10,0.0,100.0,100.0
a3b44eba-a135-11e9-9cda-0a87ae2ba916,0,10,10,10,0.0,100.0,100.0
1571105068000485406,10,10,10,10,100.0,100.0,100.0
a3acc649-a135-11e9-9cda-0a87ae2ba916,0,10,10,10,0.0,100.0,100.0
1448642162257565290,10,10,10,10,100.0,100.0,100.0
a3b4a05c-a135-11e9-9cda-0a87ae2ba916,0,10,10,10,0.0,100.0,100.0
1582242323927553626,10,10,10,10,100.0,100.0,100.0


In [23]:
# top ten worst stations
station_summary.sort_values('pct_no_classic', ascending=False).head(10)

,no_classic_count,no_docks_count,problem_count,total_count,pct_no_classic,pct_no_docks,pct_problem
station_id,,,,,,,
1594046443937813244,10,7,10,10,100.0,70.0,100.0
1594046413873042136,10,0,10,10,100.0,0.0,100.0
1594041781399920506,10,0,10,10,100.0,0.0,100.0
1594041785694887804,10,0,10,10,100.0,0.0,100.0
1594041785694887806,10,0,10,10,100.0,0.0,100.0
1594046362333434512,10,0,10,10,100.0,0.0,100.0
1594046370923369112,10,0,10,10,100.0,0.0,100.0
1594046370923369114,10,0,10,10,100.0,0.0,100.0
1594046379513303716,10,0,10,10,100.0,0.0,100.0
